4)	Реализовать автоматическое определение типа столбцов по хранящимся в таблице значениям. Оформить как отдельную функцию и встроить этот функционал как опцию работы функции load_table. 
Сложность 1 или 2
6)	Добавить набор функций add, sub, mul, div, которые обеспечат выполнение арифмитических операций для столбцов типа int, float, bool. Продумать сигнатуру функций и изменения в другие функции, которые позволят удобно выполнять арифметические операции со столбцами и присваивать результаты выч. Реализовать реагирование на некорректные значения с помощью генерации исключительных ситуаций.
Сложность 2

7)	По аналогии с п. 6 реализовать функции eq (==), gr (>), ls (<), ge (>=), le (<=), ne (==), которые возвращают список булевских значений длинной в количество строк сравниваемых столбцов. Реализовать функцию filter_rows (bool_list, copy_table=False) – получение новой таблицы из строк для которых в bool_list (длинной в количество строк в таблице) находится значение True.
Сложность 3


In [7]:
import csv
import copy
import operator
import functools

class Table:
    """Класс Table используется для загрузки данных из csv файлов

    Основное применение - редактирование существующих csv файлов

    Note:
        В первой строке файла должны находиться заголовки столбцов
        Тип колонки определяется по значению во второй строке
        Поддерживымые типы значений элементов таблицы: bool, int, float, str
    
    Attributes
    ----------
    filename : str
        полный путь до csv файла
    setTypes : bool
        Выполнять ли типизирование данных csv файла, или оставить все значения в строковом виде
    data : list of lists
        Таблица данных, в первой строке заголовки колонок, данные начинаются со второй строки

    Methods
    -------
    load_table(filename)
        Возвращает значения ячеек csv файла с адресом filename в виде списка списков
    save_table(filename)
        Записывает значения из поля data в файл csv с адресом filename
    __str__()
        Возвращает строковое представление поля data
    __getitem__(item)
        Позволяет обращаться к классу как к списку используя синтаксис []
        Возвращает data[item]
    get_rows_by_number(start, stop=None, copy_table=False):
        Возвращает строки таблицы с номерами от start до stop (не включая)
    get_rows_by_index(self, vals, copy_table=False):
        Возвращает строки таблицы в первой колонке которых содержится какое-нибудь значение из списка vals
    get_column_types(by_number=True):
        Возвращает словарь типов значений в колонках (тип определяется по второй строке таблицы)
        Ключи словаря это порядковый номер колонки отсчёт начинается с 0
        Если by_number=False ключи словаря название колонок из первой строки
    getType(val):
        Определяет тип переданного значения val и возвращает одну из строк: "bool", "int", "float", "str"
    setType(typeName, val):
        Возвращает преобразованное к типу typeName ("bool", "int", "float", "str") значение val
    tableSetType()
        Типизирует данные в таблице объекта, типы определяются по второй строке
    get_values(column=0)
        Возвращает все значения в указанной колонке (без заголовка колонки)
        Колонку можно указать по её заголовку или по-порядковому номеру
    get_value(column=0)
        Возвращает первое значение (из второй строки) в указанной колонке (без заголовка колонки)
        Колонку можно указать по её заголовку или по-порядковому номеру
    set_values(values, column=0)
        Задаёт значения values для указанной колонки (заголовок колонки не меняется)
        Колонку можно указать по её заголовку или по-порядковому номеру
        Если значений больше чем строк данных в таблице, то лишниее значения не учитываются
        Если значений меньше чем строк данных в таблице, тогда только верхние строки получат значения
    set_value(value, column=0)
        Задаёт значение value для указанной колонки второй строки
        Колонку можно указать по её заголовку или по-порядковому номеру
    print_table()
        Вызывает метод print для объекта
    execute(command,column,start, stop=None)
        Выполняет команду command для значений в колонке column для строк с номерами от start до stop (не включая)
        Колонку можно указать по её заголовку или по-порядковому номеру
        Если stop не указан - строки от start и до конца таблицы
        Доступны следующие команды: 
        "mul" перемножение элементов в колонке
        "add" суммирование элементов в колонке
        "sub" вычитание элементов колонки
        "div" целочисленное деление элементов в колонке
    filter_rows(command,value,column,start, stop=None,copy_table=False)
        Возвращает строки с номерами от start до stop (не включая) в колонке column 
        отфильтрованные в соответствии с указанной командой command и значением value
        Колонку можно указать по её заголовку или по-порядковому номеру
        Если stop не указан - строки от start и до конца таблицы
        Доступны следующие команды: 
        "eq" значение в колонке равно (==) значению value
        "ne" значение в колонке не равно (!=) значению value
        "gt" значение в колонке больше (>) значения value
        "lt" значение в колонке меньше (<) значения value
        "ge" значение в колонке больше или равно (>=) значению value
        "le" значение в колонке меньше или равно (<=) значению value
    """
    def __init__(self, filename=None, setTypes=False):
        self.filename = filename
        self.data = self.load_table(filename) if filename else []
        if setTypes:
            self.tableSetType()

    def load_table(self, filename):
        """Метод для загрузки csv файла в список списков

        Parameters
        ----------
        filename : str
            путь к файлу

        Raises
        ------
        Exception
            Если filename указан неверно вызовется исключение
        """
        
        self.filename = filename
        return [fields for fields in csv.reader(open(filename, newline=''), delimiter=";")]

    def save_table(self, filename):
        """Метод для сохранения списка списков в csv файл

        Parameters
        ----------
        filename : str
            путь к файлу

        Raises
        ------
        Exception
            Если filename указан неверно вызовется исключение
        """
        
        csv.writer(open(filename, "w", newline=''), delimiter=";").writerows(self.data)

    def __str__(self):
        """Метод возвращает строковое представление поля data"""

        return str(self.data)

    def __getitem__(self, item):
        """Позволяет обращаться к классу как к списку используя синтаксис []
        Возвращает data[item]

        Parameters
        ----------
        item : int, slice
            индекс или срез
        
        Raises
        ------
        Exception
            Если item указан неверно вызовется исключение
        """
        return self.data[item]

    def get_rows_by_number(self, start, stop=None, copy_table=False):
        """Возвращает строки таблицы с номерами от start до stop (не включая)
        Если stop не указан - строки от start и до конца таблицы

        Parameters
        ----------
        start : int
            с какой строки начать
        stop : int, None
            на какой строке закончить
        copy_table : bool
            выполнить ли глубокое копирование данных
            Если False дальнейшие изменения в возвращаемых данных будут влиять на данные объекта
        """
        if stop is None:
            stop = len(self.data)

        return copy.deepcopy(self.data[start:stop]) if copy_table else self.data[start:stop]

    def get_rows_by_index(self, vals, copy_table=False):
        """Возвращает строки таблицы в первой колонке которых содержится какое-нибудь значение из списка vals

        Parameters
        ----------
        vals : []
            список значений для поиска
        copy_table : bool
            выполнить ли глубокое копирование данных
            Если False дальнейшие изменения в возвращаемых данных будут влиять на данные объекта
        
        Raises
        ------
        Exception
            Если data содержит пустой список вызовется исключение
        """
        
        result = []

        for val in vals:
#             [1:]
            for row in self.data[1:]: 
                if row[0] == val:
                    result.append(row)

        return copy.deepcopy(result) if copy_table else result

    def get_column_types(self, by_number=True):
        """Возвращает словарь типов значений в колонках (тип определяется по второй строке таблицы)
        Ключи словаря это порядковый номер колонки отсчёт начинается с 0
        Если by_number=False ключи словаря название колонок из первой строки

        Parameters
        ----------
        by_number : bool
            определяет значения ключей словаря
        
        Raises
        ------
        IndexError
            Если data не содержит ниодного списка вызовется исключение
            и метод возвратит пустой словарь
        """
        result = {}

        for i in range(len(self.data[0])):
            try:
                columnType = self.getType(self.data[1][i])
            except IndexError:
                print("Error: в таблице нет данных")
                break

            if by_number:
                result[i] = columnType
            else:
                result[self.data[0][i]] = columnType

        return result

    def getType(self, val):
        """Определяет тип переданного значения val и возвращает одну из строк: "bool", "int", "float", "str"
        Если val == "False" или val == "True" -> "bool"
        иначе если int(val) -> "int"
        иначе если float(val) -> "float"
        иначе -> "str"

        Parameters
        ----------
        val : str
            переданное значение
        
        """
        columnType = None
        try:
            if not (val == "False" or val == "True"):
                raise TypeError()
            columnType = "bool"
        except:
            try:
                int(val)
                columnType = "int"
            except:
                try:
                    float(val)
                    columnType = "float"
                except:
                    columnType = "str"

        return columnType

    def setType(self, typeName, val):
        """Возвращает преобразованное к типу typeName ("bool", "int", "float", "str") значение val

        Parameters
        ----------
        val : str
            переданное значение
        typeName: "bool", "int", "float", "str"
            требуемый тип
        
        Raises
        ------
        Exception
            Если typeName содержит другое значение кроме "bool", "int", "float", "str" 
            или val невозможно првести к указанному типу
            вызовется исключение
        """
        return {"bool": bool,
                "int": int,
                "float": float,
                "str": str}[typeName](val)

    def tableSetType(self):
        """Типизирует данные в таблице объекта, типы определяются по второй строке

        Raises
        ------
        IndexError
            Если data не содержит данных кроме заголовков колонок вызовется исключение
        """
        for j in range(len(self.data[0])):

            try:
                columnType = self.getType(self.data[1][j])
            except IndexError:
                print("Error: в таблице нет данных")
                break

            for i in range(1, len(self.data)):
                self.data[i][j] = self.setType(columnType, self.data[i][j])

    def get_values(self, column=0):
        """Возвращает все значения в указанной колонке (без заголовка колонки)
        Колонку можно указать по её заголовку или по-порядковому номеру

        Parameters
        ----------
        column : int, str
            Определяет из какой колонки брать значения
        
        Raises
        ------
        Exception:
            Если data пустой список или если индекс колонки указан неверно вызовется исключение
        IndexError
            Если data не содержит данных кроме заголовков колонок вызовется исключение
            и метод возвратит пустой список
        """
        if isinstance(column, str):
            column = self.data[0].index(column)

        result = []
        try:
            columnType = self.getType(self.data[1][column])
        except IndexError:
            print("Error: в таблице нет данных")
            return result

        for row in range(1, len(self.data)):
            result.append(self.setType(columnType, self.data[row][column]))

        return result

    def get_value(self, column=0):
        """Возвращает первое значение (из второй строки) в указанной колонке (без заголовка колонки)
        Колонку можно указать по её заголовку или по-порядковому номеру

        Parameters
        ----------
        column : int, str
            Определяет из какой колонки брать значения
        
        Raises
        ------
        Exception:
            Если data пустой список или если индекс колонки указан неверно вызовется исключение
        IndexError
            Если data не содержит данных кроме заголовков колонок вызовется исключение
            и метод возвратит None
        """
        try:
            return self.get_values(column)[0]
        except IndexError:
            print("Error: в таблице нет данных")
#             [] None
            return None

    def set_values(self, values, column=0):
        """Задаёт значения values для указанной колонки (заголовок колонки не меняется)
        Колонку можно указать по её заголовку или по-порядковому номеру
        Если значений больше чем строк данных в таблице, то лишниее значения не учитываются
        Если значений меньше чем строк данных в таблице, тогда только верхние строки получат значения

        Parameters
        ----------
        values : []
            список задаваемых значений
        column : int, str
            Определяет из какой колонки брать значения
        
        Raises
        ------
        Exception:
            Если data пустой список или если индекс колонки указан неверно вызовется исключение
            Если data не содержит данных в указанной колонке вызовется исключение
        """
        if isinstance(column, str):
            column = self.data[0].index(column)

        minrows = min(len(values), len(self.data) - 1)

        for i in range(minrows):
            self.data[i + 1][column] = values[i]

    def set_value(self, value, column=0):
        """Задаёт значение value для указанной колонки второй строки
        Колонку можно указать по её заголовку или по-порядковому номеру

        Parameters
        ----------
        value : bool, int, float, str
            задаваемое значений
        column : int, str
            Определяет из какой колонки брать значения
        
        Raises
        ------
        Exception:
            Если data пустой список или если индекс колонки указан неверно вызовется исключение
            Если data не содержит данных в указанной колонке вызовется исключение
        """
        if isinstance(column, str):
            column = self.data[0].index(column)

        if len(self.data) - 1:
            self.data[1][column] = value

    def print_table(self):
        """Вызывает метод print для объекта"""
        print(self)

    def execute(self,command,column,start, stop=None):
        """Выполняет команду command для значений в колонке column для строк с номерами от start до stop (не включая)
        Колонку можно указать по её заголовку или по-порядковому номеру
        Если stop не указан - строки от start и до конца таблицы
        Доступны следующие команды: 
        "mul" перемножение элементов в колонке
        "add" суммирование элементов в колонке
        "sub" вычитание элементов колонки
        "div" целочисленное деление элементов в колонке

        Parameters
        ----------
        command : "mul", "add", "sub", "div"
            выполняемая команда
        column : int, str
            Определяет из какой колонки брать значения
        start : int
            с какой строки начать
        stop : int, None
            на какой строке закончить
            
        Raises
        ------
        Exception:
            Если data пустой список или если индекс колонки указан неверно вызовется исключение
            Если data не содержит данных в указанной колонке вызовется исключение
            Если к значению в колонке нельзя применить указанную команду command вызовется исключение
        IndexError:
            Если в data нет данных кроме заголовков колонок вызовется исключение
            и метод возвратит None
        """
        
        if isinstance(column, str):
            column = self.data[0].index(column)

        if stop is None:
            stop = len(self.data)

        try:
            data = [row[column] for row in self.data[start:stop]]
            if not data:
                raise (IndexError)
        except IndexError:
            print("Error: в таблице нет данных")
            return None

        return functools.reduce(*{"mul": (operator.mul, data, 1),
                                  "add": (operator.add, data),
                                  "sub": (operator.sub, data),
                                  "div": (operator.floordiv, data)}[command])

    def filter_rows(self,command,value,column,start, stop=None,copy_table=False):
        """Возвращает строки с номерами от start до stop (не включая) в колонке column 
        отфильтрованные в соответствии с указанной командой command и значением value
        Колонку можно указать по её заголовку или по-порядковому номеру
        Если stop не указан - строки от start и до конца таблицы
        Доступны следующие команды: 
        "eq" значение в колонке равно (==) значению value
        "ne" значение в колонке не равно (!=) значению value
        "gt" значение в колонке больше (>) значения value
        "lt" значение в колонке меньше (<) значения value
        "ge" значение в колонке больше или равно (>=) значению value
        "le" значение в колонке меньше или равно (<=) значению value

        Parameters
        ----------
        command : "eq", "ne", "gt", "lt", "ge", "le"
            выполняемая команда
        value : bool, int, float, str
            задаваемое значений
        column : int, str
            Определяет из какой колонки брать значения
        start : int
            с какой строки начать
        stop : int, None
            на какой строке закончить
        copy_table : bool
            выполнить ли глубокое копирование данных
            Если False дальнейшие изменения в возвращаемых данных будут влиять на данные объекта
            
        Raises
        ------
        Exception:
            Если data пустой список или если индекс колонки указан неверно вызовется исключение
            Если data не содержит данных в указанной колонке вызовется исключение
            Если к значению в колонке нельзя применить указанную команду command вызовется исключение
        """
        if isinstance(column, str):
            column = self.data[0].index(column)

        if stop is None:
            stop = len(self.data)

        operators ={"eq":operator.eq,"ne":operator.ne,"gt":operator.gt, "lt":operator.lt, "ge":operator.ge,"le":operator.le}
        rows = []
        rows.append(copy.deepcopy(self.data[0]) if copy_table else self.data[0])
        for row in self.data[start:stop]:
            if operators[command](row[column],value):
                rows.append(copy.deepcopy(row) if copy_table else row)

        return rows

In [8]:
from Table.Table import Table
f1 = "classmates.csv"
f2 = "class1.csv"

In [6]:
# загрузка файла f1
t1 = Table(f1)
print(t1)
# сохранение файла в class.csv
t1.save_table("class.csv")
print("---------------------------")
# получение строк с 1 по 3( не включая)
# copy_table=False
# изменения в t2 влияют на исходную таблицу t1
t2=t1.get_rows_by_number(1,3)
t2[0][0]='Петр'
print(t2)
print(t1)

# copy_table=True
# изменения в t2 не влияют на исходную таблицу t1
print("---------------------------")
t2=t1.get_rows_by_number(1,3,True)
t2[0][0]='Сергей'
print(t2)
print(t1)

[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Токарь', '1997'], ['Сергей', 'Сварщик', '1985'], ['Петр', 'Учитель', '1981']]
---------------------------
3
[['Петр', 'Токарь', '1997'], ['Сергей', 'Сварщик', '1985']]
[['Имя', 'Профессия', 'Год рождения'], ['Петр', 'Токарь', '1997'], ['Сергей', 'Сварщик', '1985'], ['Петр', 'Учитель', '1981']]
---------------------------
3
[['Сергей', 'Токарь', '1997'], ['Сергей', 'Сварщик', '1985']]
[['Имя', 'Профессия', 'Год рождения'], ['Петр', 'Токарь', '1997'], ['Сергей', 'Сварщик', '1985'], ['Петр', 'Учитель', '1981']]


In [5]:
# получение строк со значениями 'Виктор' или 'Петр' в первой колонке
# copy_table=False
# изменения в t2 влияют на исходную таблицу t1
t1 = Table(f1)
t2=t1.get_rows_by_index(['Виктор', 'Петр'])
print(t2)
t2[0][0]='Сергей'
t2[1][-1]='1990'
print(t1)

[['Виктор', 'Токарь', '1997'], ['Петр', 'Учитель', '1981']]
[['Имя', 'Профессия', 'Год рождения'], ['Сергей', 'Токарь', '1997'], ['Сергей', 'Сварщик', '1985'], ['Петр', 'Учитель', '1990']]


In [6]:
# получение строк со значениями 'Виктор' или 'Петр' в первой колонке
# copy_table=True
# изменения в t2 не влияют на исходную таблицу t1
t1 = Table(f1)
t2=t1.get_rows_by_index(['Виктор', 'Петр'],True)
print(t2)
t2[0][0]='Сергей'
t2[1][-1]='1990'
print(t1)

[['Виктор', 'Токарь', '1997'], ['Петр', 'Учитель', '1981']]
[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Токарь', '1997'], ['Сергей', 'Сварщик', '1985'], ['Петр', 'Учитель', '1981']]


In [7]:
# получение типов колонок (str, int, float, bool) тип определяется по второй строке
# by_number=True
# ключи словаря типов это порядковый номер колонки
print(t1.get_column_types())
# by_number=False
# ключи словаря типов это имена колонок (первая строка таблицы)
print(t1.get_column_types(False))

{0: 'str', 1: 'str', 2: 'int'}
{'Имя': 'str', 'Профессия': 'str', 'Год рождения': 'int'}


In [8]:
# Получение значений таблицы из колонки с первым порядковым номером (вторая колонка)
print(t1.get_values(1))
# Получение значений таблицы из колонки которая называется 'Год рождения'
print(t1.get_values('Год рождения'))

['Токарь', 'Сварщик', 'Учитель']
[1997, 1985, 1981]


In [9]:
# Получение первого значения из таблицы (вторая строка таблицы) из колонки с первым порядковым номером (вторая колонка)
print(t1.get_value(1))
# Получение первого значения из таблицы (вторая строка таблицы) из колонки которая называется 'Год рождения'
print(t1.get_value('Год рождения'))

Токарь
1997


In [10]:
t1 = Table(f1)
# изменить значения в таблице на заданные ('Пекарь','Инженер','Сварщик') в колонке с первым порядковым номером (вторая колонка)
t1.set_values(['Пекарь','Инженер','Сварщик'],1)
print(t1)
# изменить значения в таблице на заданные ('Фельдшер','Бухгалтер') в колонке с первым порядковым номером (вторая колонка)
t1.set_values(['Фельдшер','Бухгалтер'],1)
print(t1)
# изменить значения в таблице на заданные ('Космонавт','Стример','Блогер','Фитнес-тренер') в колонке которая называется 'Профессия'
t1.set_values(['Космонавт','Стример','Блогер','Фитнес-тренер'],'Профессия')
print(t1)
# изменить первое значение в таблице (вторая строка) на заданное ('Альпинист') в колонке которая называется 'Профессия'
t1.set_value('Альпинист','Профессия')
print(t1)
t1.print_table()

[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Пекарь', '1997'], ['Сергей', 'Инженер', '1985'], ['Петр', 'Сварщик', '1981']]
[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Фельдшер', '1997'], ['Сергей', 'Бухгалтер', '1985'], ['Петр', 'Сварщик', '1981']]
[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Космонавт', '1997'], ['Сергей', 'Стример', '1985'], ['Петр', 'Блогер', '1981']]
[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Альпинист', '1997'], ['Сергей', 'Стример', '1985'], ['Петр', 'Блогер', '1981']]
[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Альпинист', '1997'], ['Сергей', 'Стример', '1985'], ['Петр', 'Блогер', '1981']]


In [11]:
t1 = Table(f1)
print(t1)
# приведение значений таблицы к четырём типам данных: str, int, float, bool
# это можно сделать при вызове конструктора класса Table(f1, True), такой пример будет ниже
t1.tableSetType()
print(t1)

[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Токарь', '1997'], ['Сергей', 'Сварщик', '1985'], ['Петр', 'Учитель', '1981']]
[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Токарь', 1997], ['Сергей', 'Сварщик', 1985], ['Петр', 'Учитель', 1981]]


In [9]:
t1 = Table(f1)
# приведение к типам: str, int, float, bool перед выполнением математеческих операций
# иначе эти операции будут выполнены над строками (или это даже может привести к выдаче исключений)
t1.tableSetType()
# сложить все значения в колонке с индексом 2 (третья колонка) начиная со второй строки (индекс строки 1) и до конца таблицы
print(t1.execute("add",2,1))
# вычесть все значения в колонке с индексом 2 (третья колонка) начиная со второй строки (индекс строки 1) и до конца таблицы
# из первого значения вычесть второе из этой разности третье и так далее
print(t1.execute("sub",2,1))
# премножить все значения в колонке с названием 'Год рождения' (третья колонка) начиная с третьей строки (индекс строки 2) и до четвёртой строки не включая её (индекс строки 3)
print(t1.execute("mul","Год рождения",2,3))
# премножить все значения в колонке с названием 'Год рождения' (третья колонка) начиная с третьей строки (индекс строки 2) и до конца таблицы
print(t1.execute("mul","Год рождения",2))
# выполнить целочисленное деление всех значений в колонке с названием 'Год рождения' (третья колонка) начиная с третьей строки (индекс строки 2) и до конца таблицы
# первое значение делиться на второе это частное делиться на 3 и так далее
print(t1.execute("div","Год рождения",2))

5963
-1969
1985
3932285
1


In [13]:
t1 = Table(f1)
# приведение к типам: str, int, float, bool перед выполнением фильтрации строк
t1.tableSetType()
# получение строк у которых в третьей колонки (индекс 2) находится значение 1985 (eq ==) 
# фильтрация происходит со второй строки (индекс 1) и до конца таблицы
print(t1.filter_rows("eq",1985,2,1))
# получение строк у которых в третьей колонки (индекс 2) находится любое значение кроме 1985 (ne !=) 
# фильтрация происходит со второй строки (индекс 1) и до конца таблицы
print(t1.filter_rows("ne",1985,2,1))
print()
# получение строк у которых в третьей колонки (индекс 2) находится значение меньшее чем 1985 (lt <) 
# фильтрация происходит со второй строки (индекс 1) и до конца таблицы
print(t1.filter_rows("lt",1985,"Год рождения",1))
# получение строк у которых в третьей колонки (индекс 2) находится значение меньшее чем 1985 или равное 1985 (le <=) 
# фильтрация происходит со второй строки (индекс 1) и до конца таблицы
print(t1.filter_rows("le",1985,"Год рождения",1))
print()
# получение строк у которых в третьей колонки (индекс 2) находится значение большее чем 1997 (gt >) 
# фильтрация происходит со второй строки (индекс 1) и до конца таблицы
print(t1.filter_rows("gt",1997,"Год рождения",1))
# получение строк у которых в третьей колонки (индекс 2) находится значение большее чем 1997 или равное 1997 (ge >=) 
# фильтрация происходит со второй строки (индекс 1) и до конца таблицы
print(t1.filter_rows("ge",1997,"Год рождения",1))

[['Имя', 'Профессия', 'Год рождения'], ['Сергей', 'Сварщик', 1985]]
[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Токарь', 1997], ['Петр', 'Учитель', 1981]]

[['Имя', 'Профессия', 'Год рождения'], ['Петр', 'Учитель', 1981]]
[['Имя', 'Профессия', 'Год рождения'], ['Сергей', 'Сварщик', 1985], ['Петр', 'Учитель', 1981]]

[['Имя', 'Профессия', 'Год рождения']]
[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Токарь', 1997]]


In [14]:
print(t1)
# получение строк у которых в третьей колонки (индекс 2) находится значение 1985 (eq ==) 
# фильтрация происходит со второй строки (индекс 1) и до конца таблицы
# copy_table=True
# изменения в полученных фильтрацией строках не влияют на исходную таблицу t1
t1.filter_rows("eq",1985,2,1,copy_table=True)[1][-1]=1996
print(t1)
# copy_table=False
# изменения в полученных фильтрацией строках влияют на исходную таблицу t1
t1.filter_rows("eq",1985,2,1)[1][-1]=1998
print(t1)

[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Токарь', 1997], ['Сергей', 'Сварщик', 1985], ['Петр', 'Учитель', 1981]]
[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Токарь', 1997], ['Сергей', 'Сварщик', 1985], ['Петр', 'Учитель', 1981]]
[['Имя', 'Профессия', 'Год рождения'], ['Виктор', 'Токарь', 1997], ['Сергей', 'Сварщик', 1998], ['Петр', 'Учитель', 1981]]


In [15]:
# Получение таблицы из файла f2 без приведения типов, все элементы таблицы имеют строковый тип (setTypes=False)
t1 = Table(f2)
print(t1)
# Получение таблицы из файла f2 c приведением типов к str, int, float, bool (setTypes=True)
t1 = Table(f2,True)
print(t1)

[['Имя', 'Профессия', 'Год рождения', 'Сертификат', 'Показатель'], ['Виктор', 'Токарь', '1997', 'True', '-0.15'], ['Сергей', 'Сварщик', '1985', 'False', '1.234'], ['Петр', 'Учитель', '1981', 'False', '10']]
[['Имя', 'Профессия', 'Год рождения', 'Сертификат', 'Показатель'], ['Виктор', 'Токарь', 1997, True, -0.15], ['Сергей', 'Сварщик', 1985, True, 1.234], ['Петр', 'Учитель', 1981, True, 10.0]]


In [16]:
# Проверка выдачи сообщений об ошибках
t1 = Table(f1)
t1.data=t1.data[:1]
print(t1)
print(t1.get_values(1))
print(t1.get_value(1))
print(t1.filter_rows("ge",1997,"Год рождения",1))
print(t1.execute("div","Год рождения",2))
t1.set_values(['Фельдшер','Бухгалтер'],1)
t1.set_value('Альпинист','Профессия')
print(t1.get_column_types(False))
t2=t1.get_rows_by_index(['Виктор', 'Петр'],True)
t2=t1.get_rows_by_number(1,3,True)

[['Имя', 'Профессия', 'Год рождения']]
Error: в таблице нет данных
[]
Error: в таблице нет данных
Error: в таблице нет данных
[]
[['Имя', 'Профессия', 'Год рождения']]
Error: в таблице нет данных
None
Error: в таблице нет данных
{}
